In [1]:
import os
import ee
import geemap
from pathlib import Path
from datetime import datetime, timedelta , date
import time
import pandas as pd
import csv
import glob
import folium
from IPython.display import display, clear_output
import shutil
import shapefile
import ipywidgets as widgets
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 

In [4]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [5]:
shp = shapefile.Reader("".join(my_button.files))
b = []
for i in range(len(shp.fields)):
    a = shp.fields[i][0]
    b.append(a)

# give the shapefile name

file_name = widgets.Dropdown(
options=b,
description='Regional category')

file_name

Dropdown(description='Regional category', options=('DeletionFlag', 'Shape_Leng', 'Shape_Area', 'ID'), value='D…

In [6]:
statics =widgets.Dropdown(
    options=['MEAN','MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'],
    value='MEAN',
    description='Statistics')

statics

Dropdown(description='Statistics', options=('MEAN', 'MAXIMUM', 'MINIMUM', 'MEDIAN', 'STD', 'VARIANCE', 'SUM'),…

In [7]:
# give the output floder and flie name
folder = FileChooser()
display(folder)

FileChooser(path='C:\Users\Yang Hsiu\Downloads\satellite python code-20220518T062225Z-001\satellite python cod…

In [8]:
states = geemap.shp_to_ee("".join(my_button.files))

dataset = ee.Image('CGIAR/SRTM90_V4')

out_dir = os.path.expanduser(folder.selected_path)
out_dem_stats = os.path.join(out_dir, folder.selected_filename+'.csv')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

geemap.zonal_statistics(dataset, states, out_dem_stats, statistics_type=statics.value, scale=None)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Yang Hsiu\Desktop\STRM\M4.csv
